# ASL YOLO Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1BwSkvGjwLQV3Ju3WMQlOvwvxQJfIlJZ0?usp=sharing)
[![Open In GitHub](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/priyanshumahey/ASL-OpenCV-Recognition)

This notebook covers how to train the YOLOV5 deep neural network using an ASL dataset taken from roboflow. From there, the goal is to use the trained model which we can implement onto opencv.

### Preparing the Model and Dataset

In [ ]:
%%capture
#^ This ignores the output of the cell

#Cloning yolov5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -U coremltools>=4.1 onnx>=1.9.0 scikit-learn==0.19.2

## Change torch to troch == 1.7.0 in requirements
## If that doesn't work, change to 
#install torch==1.11,
#torchvision==0.12,
#torchtext==0.12,
#torchaudio==0.11

In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
%%capture

#This installs the dataset itself
%cd /content
!curl -L "https://public.roboflow.com/ds/ZOEdOs1dLv?key=4ZGGms5agy" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## If prompted, it may want you to replace the files you already have may require you to select if you want to replace them.
## [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Setup

We'll start off by figuring out exactly what we'd like to import. From there, we'll set up pytorch and make CUDA is active. On Google Colab, we'd want to switch runtime to GPU to significantly speed up the entire process.

In [ ]:
import torch
import yaml
from IPython.display import Image, clear_output
from IPython.core.magic import register_line_cell_magic
import glob
from IPython.display import Image, display


print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

We'll then want to import in data.yaml to access what the dataset actually looks like and gain a deeper look into the structure of the dataset at hand.

In [ ]:
%cat data.yaml

In [ ]:
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
with open("data.yaml", 'r') as stream:
    names = str(yaml.safe_load(stream)['names'])

print('num_classes: %s' %num_classes)
print('names: %s' %names)

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

### Using the YOLO Model

We'll be using a custom version of the yolov5 structure and here we simply create a new template file for the yolov5 algorithm to run off of and make it into a yaml file.

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

Now we'll actually train the actual yolov5 network using the dataset. We add in all the arguments we'd like to use and then train it. On Google Colab with GPU, this roughly takes 30 minutes.

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 100 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

### Looking at the Results

From there, we can load in tensorboard to view the results of the training.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

We'll also view some regular graphs

In [ ]:
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

We can also then view the results of the program by viewing what the real labels look like and then compare that to some predictions made by the model.

In [ ]:
!cd ./
!pwd

In [ ]:
#The actual real data
print("Real Labels")
Image(filename='/content/yolov5/runs/train/yolov5s_results/val_batch2_labels.jpg', width=900)

In [ ]:
#Predictions on the data
print("Predicted Labels")
Image(filename='/content/yolov5/runs/train/yolov5s_results/val_batch2_pred.jpg', width=900)

### Exporting the Weights

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
!python export.py --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt --img 640 --batch 1 --include 'onnx' # export at 640x640 with batch size 1 

In [ ]:
best_weights = "/content/yolov5/runs/train/yolov5s_results/weights/best.pt"

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ../test/images

In [ ]:
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

### Testing on your own images

Under `/content/yolov5/runs/detect/exp`, you'll need to put down the images you want detected (in jpg format)

In [ ]:
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
for imageName in glob.glob('/content/yolov5/runs/detect/exp7/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")